In [1]:
#importing the dataset
import csv
with open('transcripts.csv', encoding='latin-1') as TED_transcript:
    transcript_csv = csv.reader(TED_transcript)
    transcripts = list(transcript_csv)
    
#Converting the csv file to a list of strings (get rid of the url) and eliminating the second column
transcript_data = [t[0] for t in transcripts]
#Sanity check
#print(type(transcript_data)) prints out list

In [ ]:
#Preprocessing the data
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

def preprocess_text_list(text_list):
    """
    Preprocesses text in a list of string
    Args:
        text_list(list[str]): a list of string

    Returns:
        processed_text(list[list(str)])): a list of string preprocessed for concatenation into a single corpus
    """
    processed_text = []
    for text in text_list:
        #use re.sub to get rid pf punctuations 
        text = re.sub('[^A-Za-z0-9 ]+', '', text)
        #tokenize the words
        tokens = word_tokenize(text)
        #get rid fo stop words using NLTK's stop word list
        filtered = [word.lower() for word in tokens if word.lower() not in stop_words]
        
        #append the filtered words to the list
        processed_text.append(filtered)
    return processed_text


preprocessed_text = preprocess_text_list(transcript_data)


TypeError: 'LazyCorpusLoader' object is not callable

In [ ]:
#Corpus creation


In [ ]:
#Word2Vec

In [ ]:
#PPMI

In [ ]:
#TF-IDF

In [ ]:
#Select word pairs

In [ ]:
#Cosine similarity

In [ ]:
#Heatmap Analysis
import sklearn

Analysis from Heatmap